# Applied Data Science Capstone Project
## This notebook will be used to complete the capstone project for the IBM Applied Data Science Capstone project course

In [2]:
import pandas as pd
import numpy as np

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


## Part 1: Importing data and Creating DataFrame

In [45]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


The data is read with the _pandas_ function _read \_ html_

In [46]:
df = df[~df.Borough.str.contains("Not assigned")]

In [47]:
df = df.groupby(['Postcode','Borough']).agg({'Neighbourhood': lambda x: ', '.join(x)})

In [48]:
df = df.reset_index()

In [51]:
df = df.loc[df['Neighbourhood'] != 'Not assigned']

In [57]:
df.shape

(102, 3)